In [3]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
from pydot import Dot, Edge, Node
from automata.fa.dfa import DFA
import numpy as np
from copy import deepcopy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from distances import  custom_distanceW
import itertools
from time import time

def data_preprocess1():

    # Read the CSV file
    df = pd.read_csv(r'C:\Users\bchan\Downloads\Dodgers\univariate\Dodgers\101-freeway-traffic.test.csv')

    # Convert the 'timestamp' column to datetime format
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Add a new column for day of the week (0 = Monday, 6 = Sunday)
    df['day_of_week'] = df['timestamp'].dt.dayofweek

    # Filter only rows corresponding to each days
    mon = df[df['day_of_week'] == 0]
    tue = df[df['day_of_week'] == 1]
    wed = df[df['day_of_week'] == 2]
    thu = df[df['day_of_week'] == 3]
    fri = df[df['day_of_week'] == 4]
    sat = df[df['day_of_week'] == 5]
    sun = df[df['day_of_week'] == 6]

    sun_list, mon_list, tue_list, wed_list, thu_list, fri_list, sat_list =[], [], [], [], [], [], []
    sun_label, mon_label, tue_label, wed_label, thu_label, fri_label, sat_label=[], [], [], [], [], [], []

    #288 data points corresponds to 1day(24hrs). 1hrs-12obs. 6hrs-72 obs
    for i in range(0, len(sun['count'].values), 288):
        #print(f'i:{i}')
        if -1 not in sun['count'].values[i:i+288] and 1 not in sun['is_anomaly'].values[i:i+288]:
            sun_list.append([sum(sun['count'].values[j:j+72]) for j in range(i, i+288, 72)])
            sun_label.append([sum(sun['is_anomaly'].values[j:j+72]) for j in range(i, i+288, 72)])
        if -1 not in mon['count'].values[i:i+288]:
            mon_list.append([sum(mon['count'].values[j:j+72]) for j in range(i, i+288, 72)])
            mon_label.append([sum(mon['is_anomaly'].values[j:j+72]) for j in range(i, i+288, 72)])
        if -1 not in tue['count'].values[i:i+288]:
            tue_list.append([sum(tue['count'].values[j:j+72]) for j in range(i, i+288, 72)])
            tue_label.append([sum(tue['is_anomaly'].values[j:j+72]) for j in range(i, i+288, 72)])
        if -1 not in wed['count'].values[i:i+288]:
            wed_list.append([sum(wed['count'].values[j:j+72]) for j in range(i, i+288, 72)])
            wed_label.append([sum(wed['is_anomaly'].values[j:j+72]) for j in range(i, i+288, 72)])
        if -1 not in thu['count'].values[i:i+288]:
            thu_list.append([sum(thu['count'].values[j:j+72]) for j in range(i, i+288, 72)])
            thu_label.append([sum(thu['is_anomaly'].values[j:j+72]) for j in range(i, i+288, 72)])
        if -1 not in fri['count'].values[i:i+288] and 1 not in fri['is_anomaly'].values[i:i+288]:
            fri_list.append([sum(fri['count'].values[j:j+72]) for j in range(i, i+288, 72)])
            fri_label.append([sum(fri['is_anomaly'].values[j:j+72]) for j in range(i, i+288, 72)])
        if -1 not in sat['count'].values[i:i+288]:
            sat_list.append([sum(sat['count'].values[j:j+72]) for j in range(i, i+288, 72)])
            sat_label.append([sum(sat['is_anomaly'].values[j:j+72]) for j in range(i, i+288, 72)])
    
    rounded_satlist=[]
    for i in sat_list:
        rounded_satlist.append([round(j/100) for j in i])

    #Create labels
    G=[]
    for i in sat_label:
        if any(i):
            G.append(1)
        else:
            G.append(0)
        
    #Adding normal sequence to increase the number of normal sequence
    #for sat
    repeat=[[3, 14, 20, 16], [3, 11, 17, 17], [4, 16, 19, 16],[3, 13, 18, 16], [4, 11, 18, 15], [4, 14, 20, 17], [4, 13, 18, 15],[4, 13, 17, 13]]
    i=0
    while i < 11:    
        for r in repeat:
            rounded_satlist.append(r)
            G.append(0)
        i+=1

    #manual labelling of sequence considering very low traffic also as anamaly
    G[0]=1
    G[13]=1

    FL_dist=rounded_satlist

    FL_dist_test=[]
    G_test=[]
    for i in range(7):
        FL_dist_test.append(rounded_satlist[i])
        G_test.append(G[i])
    for i in range(len(rounded_satlist)-15, len(rounded_satlist)):
        FL_dist_test.append(rounded_satlist[i])
        G_test.append(G[i])

    Lst=[]
    Lst_test=[]
    for i in FL_dist:
        Lst.append(str(i)[1:-1].replace(' ',''))
    for i in FL_dist_test:
        Lst_test.append(str(i)[1:-1].replace(' ',''))

    FL = [l.split(',') for l in Lst]
    FL_test = [l.split(',') for l in Lst_test]

    uniq_list=set(Lst)
    alphabet = set(item for string in uniq_list for item in string.split(",") if item != ',')

    Pref_S = set()
    for string in uniq_list:
        prefixes = string.split(',')
        for i in range(1, len(prefixes) + 1):
            prefix = ','.join(prefixes[:i])
            Pref_S.add(prefix)
    Pref_S.add('')
    

    dist = custom_distanceW(np.array(FL_dist))
    #print(f'Max Distance: {np.max(dist)}')
    #np.savetxt('C:/Users/bchan/Downloads/result_TC.txt', dist, fmt='%.0f')

    return alphabet, Pref_S, Lst, FL, dist, FL_test, G_test

def train(n, eps, eps1, eps2):

    alphabet, Pref_S, Lst, FL, dist, FL_test, G_test = data_preprocess1()
    
    states = {str(f'q{i}') for i in range(n)}
    start_state = 'q0'

    env=gp.Env(empty=True)
    env.setParam('OutputFlag', 0)
    env.start()
    
    #t0 = time()
    # Creating a new model
    mtc = gp.Model("DFA_TC", env=env)

    t0 = time()

    #DECISION VARIABLES
    delta = mtc.addVars(states, alphabet, states, vtype=gp.GRB.BINARY, name='delta')
    x = mtc.addVars(Pref_S, states, vtype=gp.GRB.BINARY, name='x')
    f = mtc.addVars(states, vtype=gp.GRB.BINARY, name='f')
    alpha = mtc.addVars(len(Lst), states, vtype=gp.GRB.BINARY, name= 'alpha')
    beta = mtc.addVars(len(Lst), states, vtype=gp.GRB.BINARY, name= 'beta')

    #OBJECTIVE
    lambda_nn = len(Lst)*(len(Lst)-1)*np.max(dist)
    #print(lambda_nn)
    print(f'eps: {eps}, eps1: {eps1}, eps2: {eps2}')
    mtc.setObjective(sum(beta[i,state1]*beta[k, state2]*dist[i,k]*(eps/lambda_nn) for i,_ in enumerate(Lst) for state1 in states for k,_ in enumerate(Lst) for state2 in states if dist[i,k] != 0) \
                    + sum(alpha[k,state2]*(eps1/len(Lst)) for k,_ in enumerate(Lst) for state2 in states) \
                    + sum(delta[state1,symbol,state2]*eps2 for state1 in states for symbol in alphabet for state2 in states if state1 != state2),\
                     gp.GRB.MINIMIZE)
    
    #AUTOMATA CONSTRAINTS
    #Constraint1
    for state0 in states:
        for symbol in alphabet:
            mtc.addConstr(sum(delta[state0,symbol,state1] for state1 in states)==1, name=f'delta[{state0},{symbol}]')

    #Constraint2
    for word in Pref_S:
        mtc.addConstr(sum(x[word,state1] for state1 in states)==1, name=f'x[{word}]')

    #Constraint3
    mtc.addConstr(x['',start_state]==1, name='initial_state')

    #Constraint4 
    for state0, word, symbol, state1 in itertools.product(states, Pref_S, alphabet, states):
        if (word + ',' + symbol) in Pref_S:
            mtc.addConstr(x[word, state0] + delta[state0, symbol, state1] - 1 <= x[word + ',' + symbol, state1], name=f'transition[{state0},{word},{symbol},{state1}]')
        if word == '' and symbol in Pref_S:
            mtc.addConstr(x[word, state0] + delta[state0, symbol, state1] - 1 <= x[symbol, state1], name=f'transition[{state0},{word},{symbol},{state1}]')

    #BOUND CONSTRAINTS
    for i, word in enumerate(Lst):
        for state1 in states:
            mtc.addConstr(alpha[i, state1] >= x[word,state1] + f[state1] -1, name=f'bound_1[{state1},{i}]')        

    for i, word in enumerate(Lst):
        for state1 in states:
            mtc.addConstr(alpha[i, state1] <= x[word,state1], name=f'bound_2[{state1},{i}]')

    for i, word in enumerate(Lst):
        for state1 in states:
            mtc.addConstr(alpha[i, state1] <= f[state1], name=f'bound_3[{state1},{i}]')
    
    #not valid MILP constraint
    '''
    for i, word in enumerate(Lst):
        for state1 in states:
            mtc.addConstr(x[word, state1] * (1-f[state1]) == beta[i, state1], name=f'bound_4[{state1},{i}]')
    '''
    for i, word in enumerate(Lst):
        for state1 in states:
            mtc.addConstr(beta[i, state1] >= x[word,state1] + (1-f[state1]) -1, name=f'bound_5[{state1},{i}]')
            mtc.addConstr(beta[i, state1] <= x[word,state1], name=f'bound_6[{state1},{i}]')
            mtc.addConstr(beta[i, state1] <= (1-f[state1]), name=f'bound_7[{state1},{i}]')            
    
    mtc.optimize()
    t1 = time()
    print("Run time", (t1-t0), "seconds")
    mtc.write(rf'C:\Users\bchan\Desktop\TUD\Thesis\model_TC_{n}.lp')

    #print('Obj Value: %g' % mtc.ObjVal)
    #print(f"Alpha: {mtc.getAttr('X', alpha)}")
    #print(f"Beta: {mtc.getAttr('X', beta)}")

    if mtc.status == 1:
        status = 'LOADED'
        print(f'DFAmodel_{n} LOADED')
        #dfa1 = DFA(states=states,input_symbols=alphabet, transitions= transition_dict, initial_state= start_state, final_states={'q0'})
    
    elif mtc.status == 2:
        print(f'DFAmodel_{n} OPTIMAL')
        status='OPTIMAL'
        transitions = mtc.getAttr('X', delta)
        t_values = [(s1,a,s2) for s1 in states for s2 in states for a in alphabet if round(transitions[s1, a, s2],0) == 1]
        #for t in t_values:
            #print(t)
        f_s = mtc.getAttr('X', f)
        final_state = {s1 for s1 in states if round(f_s[s1],0) == 1}

        transition_dict = create_transition_dict(states, alphabet, t_values)
        #print(transition_dict)

        dfa1 = DFA(states=states,input_symbols=alphabet, transitions= transition_dict, initial_state= start_state, final_states=final_state)
        accepted = 0
        rejected = 0
        for w in FL:
            if dfa1.accepts_input(w):
                #print(f'{w}:accepted')
                accepted += 1             
            else:
                #print(f'{w}:rejected')
                rejected += 1        
        print(f'Accepted in Training:{accepted}')
        print(f'Rejected in Training:{rejected}')

        create_diagram(rf'C:\Users\bchan\Desktop\TUD\Thesis\diagram_TC_{n}.png', states, start_state,final_state, transition_dict)
        return dfa1, FL_test, G_test      
    
    elif mtc.status == 3:
        status = 'INFEASIBLE'
        print(f'DFAmodel_{n} INFEASIBLE')
    else:
        print('status unknown, DEBUG!!')    
 
    return status

def create_transition_dict(states, alphabet, t_values):
    transition_dict = {}

    for state in states:
        transition_dict[state] = {}
        for symbol in alphabet:
            transition_dict[state][symbol] = None

    for trans in t_values:
        current_state, symbol, next_state = trans
        transition_dict[current_state][symbol] = next_state

    return transition_dict

def create_diagram(path, states, start_state, final_state, transition_dict):

    graph = Dot(graph_type='digraph', rankdir='LR')
    nodes = {}
    for state in states:
        if state == start_state:
            # color start state with green
            if state in final_state:
                initial_state_node = Node(
                    state,
                    style='filled',
                    peripheries=2,
                    fillcolor='#66cc33')
            else:
                initial_state_node = Node(
                    state, style='filled', fillcolor='#66cc33')
            nodes[state] = initial_state_node
            graph.add_node(initial_state_node)
        else:
            if state in final_state:
                state_node = Node(state, peripheries=2)
            else:
                state_node = Node(state)
            nodes[state] = state_node
            graph.add_node(state_node)
    # adding edges
    for from_state, lookup in transition_dict.items():
        for to_label, to_state in lookup.items():
            graph.add_edge(Edge(
                nodes[from_state],
                nodes[to_state],
                label=to_label
            ))
    if path:
        graph.write_png(path)
    return graph

def test(G_test, FL_test, dfa1):
    accepted = 0
    rejected = 0
    Predicted_labels=[]

    for w in FL_test:
        #print(w)
        if dfa1.accepts_input(w):
            Predicted_labels.append(1)
            #print(f'{w}:accepted')
            accepted += 1             
        else:
            #print(f'{w}:rejected')
            Predicted_labels.append(0)
            rejected += 1  
    
    print(f'Accepted in Testing:{accepted}')
    print(f'Rejected in Testing:{rejected}')
    #print(f'Predicted_labels:{Predicted_labels}')
    #print(f'True_labelssssss:{G_test}')
    
    accuracy = accuracy_score(G_test, Predicted_labels)
    print(f'Accuracy:{round(accuracy,2)}')
    f1 = f1_score(G_test, Predicted_labels, average='binary', pos_label=1)
    print(f'F1_score:{round(f1,2)}\n')

In [ ]:
# n denotes the number of states
# eps, eps1, eps2 are regularization parameter
for n in range(2,3):
    for i, j, k in itertools.product([2], [1], [0]):
        dfa1, FL_test, G_test = train(n, eps=i, eps1=j, eps2=k)
        test(G_test, FL_test, dfa1)
    